# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234279439029                   -0.50    7.0         
  2   -7.249857121995       -1.81       -1.40    1.0   6.97ms
  3   -7.251127084059       -2.90       -1.94    2.0   8.18ms
  4   -7.251268808383       -3.85       -2.16    2.0   8.69ms
  5   -7.251315944218       -4.33       -2.46    1.0   7.09ms
  6   -7.251333441167       -4.76       -2.80    1.0   7.10ms
  7   -7.251338396665       -5.30       -3.37    1.0   7.19ms
  8   -7.251338752751       -6.45       -3.76    2.0   8.51ms
  9   -7.251338795722       -7.37       -4.42    2.0   8.54ms
 10   -7.251338798484       -8.56       -4.96    3.0   9.77ms
 11   -7.251338798674       -9.72       -5.34    2.0   9.18ms
 12   -7.251338798699      -10.60       -5.74    2.0   8.72ms
 13   -7.251338798704      -11.34       -6.24    2.0   8.78ms
 14   -7.251338798704      -12.19       -6.53    3.0   10.5ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.0787808194888326
[ Info: Arnoldi iteration step 2: normres = 0.6385327821114197
[ Info: Arnoldi iteration step 3: normres = 0.70457963707631
[ Info: Arnoldi iteration step 4: normres = 0.27211796389115955
[ Info: Arnoldi iteration step 5: normres = 0.4785777278058789
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.92e-02, 7.72e-02, 3.61e-01, 3.04e-01, 8.97e-03)
[ Info: Arnoldi iteration step 6: normres = 0.2754205416183754
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.70e-03, 1.57e-01, 1.91e-01, 7.83e-02, 8.43e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0741549725232667
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.63e-04, 1.07e-02, 1.02e-02, 2.31e-02, 5.90e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11591967518002157
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (8.14e-06, 